# Implementação de Métodos de Otimização

Vamos começar implementando um método simples.

## Método do Gradiente com busca de Armijo

$$ x^{k+1} = x^k + t_kd^k $$
$$ f(x^{k+1}) < f(x^k) + \alpha t^k \nabla f(x^k)^Td^k $$

In [90]:
function metodo_gradiente(f, ∇f, x₀)
    ϵ = 1e-4
    α = 0.5
    η = 0.5
    kmax = 1000
    
    ef = 0
    x = copy(x₀)
    k = 0
    while norm(∇f(x)) > ϵ
        d = -∇f(x)
        t = 1.0
        while f(x + t*d) > f(x) + α*t*dot(d,∇f(x))
            t *= η
        end
        x = x + t*d
        k += 1
        if k > kmax
            ef = 1
            break
        end
    end
    return x, f(x), ef, k
end

metodo_gradiente (generic function with 1 method)

In [93]:
n = 100
Λ = linspace(1e-4, 1.0, n) # Linearly spaced vector from 1e-4 to 1.0
f(x) = 0.5*dot(x, Λ.*x); ∇f(x) = Λ.*x

x, fx, ef, k = metodo_gradiente(f, ∇f, ones(n))

([0.945442,0.00317775,1.00747e-5,3.00921e-8,8.45733e-11,2.23366e-13,5.53637e-16,1.28606e-18,2.79584e-21,5.67987e-24  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],4.4744523760589496e-5,0,561)

In [96]:
println("f(x) = $fx; ef = $ef; k = $k")

f(x) = 4.4744523760589496e-5; ef = 0; k = 561


### Upgrade da função

Uma maneira simples de limpar a função é colocar alguns parâmetros como
argumentos por palavra-chave. Assim já temos a opção de fazer mudanças na
chamada da função, mas também temos uma opção simples para outras pessoas usarem.
Se tivermos muitas funções com muitos argumentos isso pode ficar complicado,
e/ou feio. Podemos tentar melhorar um pouco a estética usando um tipo
composto (equivalente à struct, mas que permite construtores).
Outra opção é usar o pacote [Options.jl](https://github.com/JuliaLang/Options.jl).

In [117]:
function metodo_gradiente2(f, ∇f, x₀; ϵ = 1e-4, α = 0.5, η = 0.5, kmax = 1000)
    ef = 0
    x = copy(x₀)
    k = 0
    while norm(∇f(x)) > ϵ
        d = -∇f(x)
        t = 1.0
        while f(x + t*d) > f(x) + α*t*dot(d,∇f(x))
            t *= η
        end
        x = x + t*d
        k += 1
        if k > kmax
            ef = 1
            break
        end
    end
    return x, f(x), ef, k
end

metodo_gradiente2 (generic function with 1 method)

In [118]:
x, fx, ef, k = metodo_gradiente2(f, ∇f, ones(n), ϵ=1e-5, kmax=100000)
println("f(x) = $fx; ef = $ef; k = $k")

f(x) = 4.99969961220827e-7; ef = 0; k = 23025


## Melhorando o código um pouco

Estamos calculando $f(x)$ e $\nabla f(x)$ toda iteração. Vamos evitar.

In [131]:
function metodo_gradiente3(f, ∇f, x₀; ϵ = 1e-4, α = 0.5, η = 0.5, kmax = 1000)
    ef = 0
    x = copy(x₀)
    k = 0
    ∇fx = ∇f(x)
    while norm(∇fx) > ϵ
        t = 1.0
        fx = f(x)
        dt∇f = dot(∇fx,∇fx)
        while f(x - t*∇fx) > fx - α*t*dt∇f
            t *= η
        end
        x = x - t*∇fx
        ∇fx = ∇f(x)
        k += 1
        if k > kmax
            ef = 1
            break
        end
    end
    return x, fx, ef, k
end

metodo_gradiente3 (generic function with 1 method)

In [136]:
x, fx, ef, k = metodo_gradiente3(f, ∇f, ones(n), ϵ=1e-5, kmax=100000)
println("f(x) = $fx; ef = $ef; k = $k")

f(x) = 4.99969961220827e-7; ef = 0; k = 23025


In [143]:
@time x, fx, ef, k = metodo_gradiente2(f, ∇f, ones(n));

  0.013025 seconds (64.01 k allocations: 6.674 MB, 26.70% gc time)


In [144]:
@time x, fx, ef, k = metodo_gradiente3(f, ∇f, ones(n));

  0.005295 seconds (42.11 k allocations: 4.532 MB)
